# nonimputed_data_summary_statistics.ipynb

**Primary Analyst:** Patrick Wu

In [1]:
!date

Mon Dec 10 10:12:15 CST 2018


In [2]:
%matplotlib inline
import numpy as np
import pymc3 as pm
import pylab as plt
import pandas as pd
import seaborn as sns
sns.set(style="darkgrid")

# Set seed
np.random.seed(42)

In [3]:
dir = '../data/'

## Aim #1: get descriptive statistics for final merged (non-imputed) table used as input for statistical models

In [4]:
pd.options.display.max_columns = None

In [5]:
# import merged table
merged_table = pd.read_csv(dir+'./merged.csv', dtype={'ruid': str})


In [6]:
merged_table1 = merged_table.copy()
merged_table1.head()

,ruid,visit_id,admit_date,discharge_date,stay_length,n_transfers,readmit_time,readmit_30d,sex,dob,dod,race,cpt_anesthesia,cpt_eval_manage,cpt_expired,cpt_medicine,cpt_modifier,cpt_path_lab,cpt_radiology,cpt_surgery,cpt_unknown,icd_dx_blood,icd_dx_circulatory,icd_dx_congenital,icd_dx_digestive,icd_dx_endocrine,icd_dx_external,icd_dx_gu,icd_dx_infection,icd_dx_injury,icd_dx_mental,icd_dx_muscskel,icd_dx_neoplasm,icd_dx_nervous,icd_dx_obstetric,icd_dx_perinatal,icd_dx_respiratory,icd_dx_skin,icd_dx_symptoms,icd_proc,icd_visit,med_Antihypertensive Agents,"med_Analgesics, Opioid",med_Narcotics,med_Antipyretics,med_Anti-Bacterial Agents,"med_Anti-Inflammatory Agents, Non-Steroidal","med_Analgesics, Non-Narcotic",med_Antiemetics,med_Diuretics,med_Anti-Arrhythmia Agents,med_Anti-Allergic Agents,med_Anti-Inflammatory Agents,med_Vasodilator Agents,med_Anti-Ulcer Agents,med_Anti-Anxiety Agents,med_Fibrinolytic Agents,med_Bronchodilator Agents,med_Antipruritics,med_Cyclooxygenase Inhibitors,med_Anticholesteremic Agents,med_Proton Pump Inhibitors,med_Hydroxymethylglutaryl-CoA Reductase Inhibitors,med_Glucocorticoids,med_Anticoagulants,med_Sympatholytics,med_Hypoglycemic Agents,med_Enzyme Inhibitors,med_Hypnotics and Sedatives,med_Sodium Potassium Chloride Symporter Inhibitors,med_Platelet Aggregation Inhibitors,med_Calcium Channel Blockers,"med_Antineoplastic Agents, Hormonal",med_Histamine H1 Antagonists,med_Anticonvulsants,med_Antipsychotic Agents,med_Immunosuppressive Agents,med_Adrenergic beta-1 Receptor Antagonists,"med_Antidepressive Agents, Second-Generation",med_Dermatologic Agents,med_GABA Modulators,med_Serotonin Uptake Inhibitors,med_Vitamins,med_Antitussive Agents,med_Angiotensin-Converting Enzyme Inhibitors,med_Antifungal Agents,med_Analgesics,med_Tocolytic Agents,med_Bone Density Conservation Agents,med_Serotonin Antagonists,med_Cardiotonic Agents,lab_an-gap_median,lab_bun_median,lab_co2_median,lab_ca_median,lab_cl_median,lab_creat_median,lab_glubed_median,lab_gluc_median,lab_hgb_median,lab_k_median,lab_mch_median,lab_mchc_median,lab_mcv_median,lab_na_median,lab_pcv_median,lab_plt-ct_median,lab_rbc_median,lab_rdw_median,lab_wbc_median,lab_an-gap_5p,lab_bun_5p,lab_co2_5p,lab_ca_5p,lab_cl_5p,lab_creat_5p,lab_glubed_5p,lab_gluc_5p,lab_hgb_5p,lab_k_5p,lab_mch_5p,lab_mchc_5p,lab_mcv_5p,lab_na_5p,lab_pcv_5p,lab_plt-ct_5p,lab_rbc_5p,lab_rdw_5p,lab_wbc_5p,lab_an-gap_95p,lab_bun_95p,lab_co2_95p,lab_ca_95p,lab_cl_95p,lab_creat_95p,lab_glubed_95p,lab_gluc_95p,lab_hgb_95p,lab_k_95p,lab_mch_95p,lab_mchc_95p,lab_mcv_95p,lab_na_95p,lab_pcv_95p,lab_plt-ct_95p,lab_rbc_95p,lab_rdw_95p,lab_wbc_95p,lab_an-gap_first_measure,lab_bun_first_measure,lab_co2_first_measure,lab_ca_first_measure,lab_cl_first_measure,lab_creat_first_measure,lab_glubed_first_measure,lab_gluc_first_measure,lab_hgb_first_measure,lab_k_first_measure,lab_mch_first_measure,lab_mchc_first_measure,lab_mcv_first_measure,lab_na_first_measure,lab_pcv_first_measure,lab_plt-ct_first_measure,lab_rbc_first_measure,lab_rdw_first_measure,lab_wbc_first_measure,lab_an-gap_last_measure,lab_bun_last_measure,lab_co2_last_measure,lab_ca_last_measure,lab_cl_last_measure,lab_creat_last_measure,lab_glubed_last_measure,lab_gluc_last_measure,lab_hgb_last_measure,lab_k_last_measure,lab_mch_last_measure,lab_mchc_last_measure,lab_mcv_last_measure,lab_na_last_measure,lab_pcv_last_measure,lab_plt-ct_last_measure,lab_rbc_last_measure,lab_rdw_last_measure,lab_wbc_last_measure,lab_an-gap_std,lab_bun_std,lab_co2_std,lab_ca_std,lab_cl_std,lab_creat_std,lab_glubed_std,lab_gluc_std,lab_hgb_std,lab_k_std,lab_mch_std,lab_mchc_std,lab_mcv_std,lab_na_std,lab_pcv_std,lab_plt-ct_std,lab_rbc_std,lab_rdw_std,lab_wbc_std,bmi_last,pregnancy_indicator_last,lab_systolic_median,lab_diastolic_median,lab_systolic_5p,lab_diastolic_5p,lab_systolic_95p,lab_diastolic_95p,lab_systolic_std,lab_diastolic_std,age,total_encounters,group
0,50135262,0,2007-02-08,2007-02-12,4,2,172 days 00:00:00.000000000,0,F,1949-09-20,2100-01-01,W,0.0,

In [7]:
# create list with column names to drop from 'merged_table1'; will calculate statistics for 'sex' and 'race' separately
drop_columns = ['ruid','visit_id','admit_date','discharge_date','dob','dod','sex','race']

In [8]:
merged_table2 = merged_table1.drop(columns=drop_columns)

In [9]:
merged_table2.shape

(21133, 210)

1) get summary statistics values for predictor variables used in `Random Forest` and `Bayesian Logistic Regression` model

In [10]:
merged_table_summary_stats = merged_table2.describe()

In [11]:
merged_table_summary_stats

,stay_length,n_transfers,readmit_30d,cpt_anesthesia,cpt_eval_manage,cpt_expired,cpt_medicine,cpt_modifier,cpt_path_lab,cpt_radiology,cpt_surgery,cpt_unknown,icd_dx_blood,icd_dx_circulatory,icd_dx_congenital,icd_dx_digestive,icd_dx_endocrine,icd_dx_external,icd_dx_gu,icd_dx_infection,icd_dx_injury,icd_dx_mental,icd_dx_muscskel,icd_dx_neoplasm,icd_dx_nervous,icd_dx_obstetric,icd_dx_perinatal,icd_dx_respiratory,icd_dx_skin,icd_dx_symptoms,icd_proc,icd_visit,med_Antihypertensive Agents,"med_Analgesics, Opioid",med_Narcotics,med_Antipyretics,med_Anti-Bacterial Agents,"med_Anti-Inflammatory Agents, Non-Steroidal","med_Analgesics, Non-Narcotic",med_Antiemetics,med_Diuretics,med_Anti-Arrhythmia Agents,med_Anti-Allergic Agents,med_Anti-Inflammatory Agents,med_Vasodilator Agents,med_Anti-Ulcer Agents,med_Anti-Anxiety Agents,med_Fibrinolytic Agents,med_Bronchodilator Agents,med_Antipruritics,med_Cyclooxygenase Inhibitors,med_Anticholesteremic Agents,med_Proton Pump Inhibitors,med_Hydroxymethylglutaryl-CoA Reductase Inhibitors,med_Glucocorticoids,med_Anticoagulants,med_Sympatholytics,med_Hypoglycemic Agents,med_Enzyme Inhibitors,med_Hypnotics and Sedatives,med_Sodium Potassium Chloride Symporter Inhibitors,med_Platelet Aggregation Inhibitors,med_Calcium Channel Blockers,"med_Antineoplastic Agents, Hormonal",med_Histamine H1 Antagonists,med_Anticonvulsants,med_Antipsychotic Agents,med_Immunosuppressive Agents,med_Adrenergic beta-1 Receptor Antagonists,"med_Antidepressive Agents, Second-Generation",med_Dermatologic Agents,med_GABA Modulators,med_Serotonin Uptake Inhibitors,med_Vitamins,med_Antitussive Agents,med_Angiotensin-Converting Enzyme Inhibitors,med_Antifungal Agents,med_Analgesics,med_Tocolytic Agents,med_Bone Density Conservation Agents,med_Serotonin Antagonists,med_Cardiotonic Agents,lab_an-gap_median,lab_bun_median,lab_co2_median,lab_ca_median,lab_cl_median,lab_creat_median,lab_glubed_median,lab_gluc_median,lab_hgb_median,lab_k_median,lab_mch_median,lab_mchc_median,lab_mcv_median,lab_na_median,lab_pcv_median,lab_plt-ct_median,lab_rbc_median,lab_rdw_median,lab_wbc_median,lab_an-gap_5p,lab_bun_5p,lab_co2_5p,lab_ca_5p,lab_cl_5p,lab_creat_5p,lab_glubed_5p,lab_gluc_5p,lab_hgb_5p,lab_k_5p,lab_mch_5p,lab_mchc_5p,lab_mcv_5p,lab_na_5p,lab_pcv_5p,lab_plt-ct_5p,lab_rbc_5p,lab_rdw_5p,lab_wbc_5p,lab_an-gap_95p,lab_bun_95p,lab_co2_95p,lab_ca_95p,lab_cl_95p,lab_creat_95p,lab_glubed_95p,lab_gluc_95p,lab_hgb_95p,lab_k_95p,lab_mch_95p,lab_mchc_95p,lab_mcv_95p,lab_na_95p,lab_pcv_95p,lab_plt-ct_95p,lab_rbc_95p,lab_rdw_95p,lab_wbc_95p,lab_an-gap_first_measure,lab_bun_first_measure,lab_co2_first_measure,lab_ca_first_measure,lab_cl_first_measure,lab_creat_first_measure,lab_glubed_first_measure,lab_gluc_first_measure,lab_hgb_first_measure,lab_k_first_measure,lab_mch_first_measure,lab_mchc_first_measure,lab_mcv_first_measure,lab_na_first_measure,lab_pcv_first_measure,lab_plt-ct_first_measure,lab_rbc_first_measure,lab_rdw_first_measure,lab_wbc_first_measure,lab_an-gap_last_measure,lab_bun_last_measure,lab_co2_last_measure,lab_ca_last_measure,lab_cl_last_measure,lab_creat_last_measure,lab_glubed_last_measure,lab_gluc_last_measure,lab_hgb_last_measure,lab_k_last_measure,lab_mch_last_measure,lab_mchc_last_measure,lab_mcv_last_measure,lab_na_last_measure,lab_pcv_last_measure,lab_plt-ct_last_measure,lab_rbc_last_measure,lab_rdw_last_measure,lab_wbc_last_measure,lab_an-gap_std,lab_bun_std,lab_co2_std,lab_ca_std,lab_cl_std,lab_creat_std,lab_glubed_std,lab_gluc_std,lab_hgb_std,lab_k_std,lab_mch_std,lab_mchc_std,lab_mcv_std,lab_na_std,lab_pcv_std,lab_plt-ct_std,lab_rbc_std,lab_rdw_std,lab_wbc_std,bmi_last,pregnancy_indicator_last,lab_systolic_median,lab_diastolic_median,lab_systolic_5p,lab_diastolic_5p,lab_systolic_95p,lab_diastolic_95p,lab_systolic_std,lab_diastolic_std,age,total_encounters
count,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,21133.000000,

2) calculate descriptive statistics for `sex` and `race`

`sex` descriptive statistics

In [12]:
sex_stats = merged_table1['sex'].value_counts()

In [13]:
sex_stats

F    10779
M    10340
U       14
Name: sex, dtype: int64

In [14]:
total_sex = 10779+10340+14
female_percentage = (10779/total_sex)*100
male_percentage = (10340/total_sex)*100

print('% Females:',female_percentage)
print('% Males:',male_percentage)

% Females: 51.0055363649269
% Males: 48.928216533383804


`race` descriptive statistics

In [15]:
race_stats = merged_table1['race'].value_counts()

In [16]:
race_stats

W    15822
B     4503
H      305
U      275
A      149
N       72
I        7
Name: race, dtype: int64

In [17]:
total_race = 15822+4503+305+275+149+72+7
white_percentage = (15822/total_race)*100
black_percentage = (4503/total_race)*100

print('% White:',white_percentage)
print('% Black:',black_percentage)

% White: 74.86868878058014
% Black: 21.307907064780203
